In [1]:
! pip install rasterio 
! pip install geopandas
! pip install rioxarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
# Working with raster data
import rasterio
import numpy as np
from rasterio.features import rasterize
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing
from shapely.geometry import Polygon
import geopandas as gpd
from shapely.geometry import box
import rioxarray
from shapely.geometry import Polygon, mapping
import rasterio.plot as plot
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import random
import json
import joblib
import pandas as pd
import datetime
from random import sample

In [23]:
shape_file = '/content/drive/MyDrive/VarunaHackathon2022/training_area/traindata.shp'
src = '/content/drive/MyDrive/VarunaHackathon2022/sentinel-2-image/'
year = '2021'

In [42]:
all_dir = os.listdir(src + year)

for j in range(len(all_dir)):

    b4 = src + year + '/' + all_dir[j] + '/IMG_DATA/47PQS_' + all_dir[j] + '_B04.jp2'
    b8 = src + year + '/' + all_dir[j] + '/IMG_DATA/47PQS_' + all_dir[j] + '_B08.jp2'

    outfile = routfile = r'/content/ndvi_temp.tif'

    #open the bands (I can't believe how easy is this with rasterio!)
    with rasterio.open(b4) as red:
        RED = red.read()
    with rasterio.open(b8) as nir:
        NIR = nir.read()

    #compute the ndvi
    # NDVI = (NIR – Red) / (NIR + Red) NIR=B8 Red=B4
    ndvi = (NIR.astype(float) - RED.astype(float)) / (NIR+RED)
    #print(np.nanmin(ndvi), np.nanmax(ndvi))

    profile = red.meta
    profile.update(driver='GTiff')
    profile.update(dtype=rasterio.float32)

    with rasterio.open(outfile, 'w', **profile) as dst:
        dst.write(ndvi.astype(rasterio.float32))

    ground_truth = gpd.read_file(shape_file)
    NDVI_dataset = rasterio.open('/content/ndvi_temp.tif')
    NDVI_dataset_p = rasterio.open('/content/ndvi_temp.tif').profile
    label = gpd.read_file(shape_file).to_crs(NDVI_dataset_p['crs'])
    all_mean = []
    for i in range(len(ground_truth)):
        polygon_geom = mapping(ground_truth.geometry[i])

        roi_polygon_src_coords = rasterio.warp.transform_geom({'init': 'epsg:32647'},
                                                    NDVI_dataset_p['crs'],                                            
                                                    polygon_geom)

        out_image,out_transform = rasterio.mask.mask(NDVI_dataset,
                                            [roi_polygon_src_coords],
                                            crop=True)

        out_image = np.clip(out_image[2::-1],
                          0,2200)/2200

        #plt.figure(figsize=(10,10))
        #plot.show(out_image,
        #          transform=out_transform)
        #all_mean.append(np.mean(out_image[out_image!=0]))

        temp = []
        temp.append(np.mean(out_image[out_image!=0]))
        temp.append(int(label['crop_type'][i]))
        all_mean.append(temp)

        all_meanTTT=np.array(all_mean).copy()
        all_meanTTT=np.transpose(all_meanTTT)
        all_meanS=list(all_meanTTT[0])
        y = list(ground_truth['crop_type'].astype(int))
        insert = list(all_meanTTT[0][all_meanTTT[1]==2])
    for i in range(len(insert)):
        all_meanS.append(sample(insert,1)[0])
        y.append(2)
    for i in range(len(insert[:30])):
        all_meanS.append(sample(insert,1)[0])
        y.append(2)
    insert = list(all_meanTTT[0][all_meanTTT[1]==3])
    for i in range(len(insert[:77])):
        all_meanS.append(sample(insert,1)[0])
        y.append(3)
    insert = list(all_meanTTT[0][all_meanTTT[1]==4])
    for i in range(len(insert)):
        all_meanS.append(sample(insert,1)[0])
        y.append(4)
    for i in range(len(insert)):
        all_meanS.append(sample(insert,1)[0])
        y.append(4)
    for i in range(len(insert[:108])):
        all_meanS.append(sample(insert,1)[0])
        y.append(4)

    # JSON file
    f = open (src + year + '/' + all_dir[j] + '/productInfo.json', "r")
    # Reading from file
    data = json.loads(f.read())
    # Iterating through the json
    # list
    data_week = pd.Timestamp(datetime.datetime.strptime(data['timestamp'],"%Y-%m-%dT%H:%M:%S.%fZ")).week
    # Closing file
    f.close()
    
    add = [data_week] * len(all_meanS)
    all_mean_t=[all_meanS]
    all_mean_t.append(add)
    all_mean_t=np.array(all_mean_t).astype('f')
    all_mean_t = np.nan_to_num(all_mean_t, nan=0)
    all_mean_t = np.transpose(all_mean_t)

    save = pd.DataFrame(all_mean_t, columns=['mean', 'week'])
    save_y = pd.DataFrame(y, columns=['crop_type'])
    save.to_csv("/content/drive/MyDrive/ARV_Hackathon/NDVI_Process_mean_time/" + all_dir[j] + ".csv")
    save_y.to_csv("/content/drive/MyDrive/ARV_Hackathon/NDVI_Process_mean_time_y/" + all_dir[j] + "_y.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encou